In [19]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam,SGD
import numpy as np
import os

from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數


# 本範例不需使用 GPU, 將 GPU 設定為 "無"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [20]:
batch_size = 32 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 2 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

# create a data generator
datagenForTrain = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

datagenForValid = ImageDataGenerator(validation_split=.2)

IMAGE_SIZE = (224, 224)


# load and iterate training dataset
train_it = datagenForTrain.flow_from_directory('../data/kaggle_dogcat/train',target_size=IMAGE_SIZE, class_mode='binary', batch_size=batch_size, subset='training')


# load and iterate validation dataset
#valid_it = datagen.flow_from_directory('../data/kaggle_dogcat/train', class_mode='binary', batch_size=batch_size, subset='validation')
valid_it = datagenForValid.flow_from_directory('../data/kaggle_dogcat/train',target_size=IMAGE_SIZE, class_mode='binary', batch_size=batch_size, subset='validation')

# confirm the iterator works
batchX = train_it.next()
batchY = valid_it.next()

# load and iterate test dataset
# test_it = datagen.flow_from_directory('../data/kaggle_dogcat/test', class_mode='binary', batch_size=64)

Found 4000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [ ]:
# 建立 ResNet 模型
resnetModel = resnet(input_shape=batchX[0][0].shape, num_classes=1) 
resnetModel.summary()

In [21]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=batchX[0][0].shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
activation_45 (Activation)   (None, 224, 224, 32)      0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 222, 222, 32)      9248      
_________________________________________________________________
activation_46 (Activation)   (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 111, 111, 64)      18496     
__________

In [22]:
#model.fit_generator(train_it, steps_per_epoch=16, validation_data=valid_it, validation_steps=8)

model.fit_generator(
    train_it,
    steps_per_epoch = train_it.samples // batch_size,
    validation_data = valid_it, 
    validation_steps = valid_it.samples // batch_size,
    epochs = epochs)


# resnetModel.compile(loss='hinge',
#               optimizer=SGD(lr=0.01, momentum=0.9),
#               metrics=['accuracy'])

# resnetModel.fit_generator(
#     train_it,
#     steps_per_epoch = train_it.samples // batch_size,
#     validation_data = valid_it, 
#     validation_steps = valid_it.samples // batch_size,
#     epochs = epochs)




Epoch 1/10
125/125 [==============================] - 731s 6s/step - loss: 7.9911 - acc: 0.4988 - val_loss: 8.0110 - val_acc: 0.4975
Epoch 2/10
125/125 [==============================] - 686s 5s/step - loss: 7.9752 - acc: 0.4997 - val_loss: 7.9114 - val_acc: 0.5038
Epoch 3/10
125/125 [==============================] - 630s 5s/step - loss: 7.9552 - acc: 0.5010 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 4/10
125/125 [==============================] - 658s 5s/step - loss: 7.9792 - acc: 0.4995 - val_loss: 7.9313 - val_acc: 0.5025
Epoch 5/10
125/125 [==============================] - 653s 5s/step - loss: 7.9831 - acc: 0.4993 - val_loss: 8.0310 - val_acc: 0.4963
Epoch 6/10
125/125 [==============================] - 653s 5s/step - loss: 7.9632 - acc: 0.5005 - val_loss: 8.0310 - val_acc: 0.4963
Epoch 7/10
125/125 [==============================] - 626s 5s/step - loss: 7.9712 - acc: 0.5000 - val_loss: 8.0110 - val_acc: 0.4975
Epoch 8/10
125/125 [==============================] - 608s 5s/step - 

In [ ]:
score = model.evaluate_generator(valid_it,steps=valid_it.samples // batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])